In [1]:
#
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import os
import seaborn as sns
from datetime import datetime, timedelta

In [2]:
# join with animal df to get calving date
# dir = os.path.join(os.path.normpath(dirpath + os.sep + os.pardir),"udder_processing",  "udder_features")
animaldf = pd.read_csv("animal_info.csv", sep = ";")
animaldf.columns = [col.replace(" ", "_").lower().replace("-", "") for col in animaldf.columns]
fresh_dates = animaldf[["animal_number", "last_calving_date", "lactation_number"]]
fresh_dates["fresh_date"] = pd.to_datetime(fresh_dates["last_calving_date"], format = '%m/%d/%Y')
fresh_dates = fresh_dates[~fresh_dates.fresh_date.isnull()]
fresh_dates = fresh_dates.drop("last_calving_date", axis = 1)
# filter records only on current lactation 
# get kickoffs current lactation per quarter

C:\Users\marie\AppData\Local\Temp\ipykernel_12544\1852963493.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fresh_dates["fresh_date"] = pd.to_datetime(fresh_dates["last_calving_date"], format = '%m/%d/%Y')


In [3]:
# filter records only on current lactation
df = pd.read_csv(r"kick_offs\kick_offs_year.csv", sep = ";")
df.columns = [col.replace(" ", "_").lower().replace("-", "") for col in df.columns]

# [re.findall("U", val) for val in df.kickoff_lr]
df["ko_lr"] = [0 if "U" in val else 1 for val in df.kickoff_lr]
df["ko_lf"] = [0 if "U" in val else 1 for val in df.kickoff_lf]
df["ko_rf"] = [0 if "U" in val else 1 for val in df.kickoff_rf]
df["ko_rr"] = [0 if "U" in val else 1 for val in df.kickoff_rr]

df = df.drop(["kickoff_rf", "kickoff_lf", "kickoff_lr", "kickoff_rr"], axis = 1)

df["date"] = pd.to_datetime(df["date"], format = '%m/%d/%Y')
merged_df =  pd.merge(fresh_dates, df,  left_on="animal_number", right_on = "animal_number")

merged_df["dim"] = (merged_df["date"] - merged_df["fresh_date"]).dt.days
merged_df["total"] = merged_df["ko_lf"] + merged_df["ko_rf"] + merged_df["ko_rr"]+ merged_df["ko_lr"]
merged_df = merged_df[merged_df.dim >= 0]

# Kick off daily

## Days with kick offs current lactation 

In [4]:
ko_df = merged_df.groupby(["animal_number", "lactation_number", "date", "dim"])[["ko_lr","ko_lf", "ko_rf","ko_rr"]].agg("sum").reset_index()
ko_df2 = ko_df.copy()
ko_df2["dko_lr"] = (ko_df["ko_lr"] > 0).astype(int)
ko_df2["dko_lf"] = (ko_df["ko_lf"] > 0).astype(int)
ko_df2["dko_rf"] = (ko_df["ko_rf"] > 0).astype(int)
ko_df2["dko_rr"] = (ko_df["ko_rr"] > 0).astype(int)

ko_df2.loc[:,"dko_udder"] = ((ko_df2.dko_rr + ko_df2.dko_rf +ko_df2.dko_lr + ko_df2.dko_lf) > 0 ).astype(int)

ko_df2.to_csv(r"data_out\ko_daily.csv", index = False)
ko_df3 = ko_df2.groupby(["animal_number", "lactation_number"]).agg({"dko_lr":"sum", "dko_lf":"sum", "dko_rr":"sum", "dko_rf":"sum", "dko_udder":"sum", "dim":"max"}).reset_index()
ko_df3 = ko_df3.rename(columns ={"dko_lr": "dko_lr_cl", "dko_lf":"dko_lf_cl", "dko_rr":"dko_rr_cl", "dko_rf":"dko_rf_cl", "dko_udder":"dko_udder_cl"})

ko_df3["dko_lr_dim_cl"] = (ko_df3["dko_lr_cl"]/ko_df3["dim"])*100
ko_df3["dko_lf_dim_cl"] = (ko_df3["dko_lf_cl"]/ko_df3["dim"])*100
ko_df3["dko_rf_dim_cl"] = (ko_df3["dko_rf_cl"]/ko_df3["dim"])*100
ko_df3["dko_rr_dim_cl"] = (ko_df3["dko_rr_cl"]/ko_df3["dim"])*100
ko_df3["dko_udder_dim_cl"] = (ko_df3["dko_udder_cl"]/ko_df3["dim"])*100

kicko1 = ko_df3

## Days with kick offs last 7d (2023-11-9 to 2023-11-16)

In [5]:
ko_df = merged_df[(merged_df.date >= '2023-11-10') & (merged_df.date < '2023-11-17')]
ko_df2["dko_lr"] = (ko_df["ko_lr"] > 0).astype(int)
ko_df2["dko_lf"] = (ko_df["ko_lf"] > 0).astype(int)
ko_df2["dko_rf"] = (ko_df["ko_rf"] > 0).astype(int)
ko_df2["dko_rr"] = (ko_df["ko_rr"] > 0).astype(int)

ko_df2.loc[:, "dko_udder"] = ((ko_df2.dko_rr + ko_df2.dko_rf +ko_df2.dko_lr + ko_df2.dko_lf) > 0 ).astype(int)
ko_df3 = ko_df2.groupby(["animal_number", "lactation_number"]).agg({"dko_lr":"sum", "dko_lf":"sum", "dko_rr":"sum", "dko_rf":"sum", "dko_udder":"sum"}).reset_index()
ko_df3 = ko_df3.rename(columns ={"dko_lr": "dko_lr_7d", "dko_lf":"dko_lf_7d", "dko_rr":"dko_rr_7d", "dko_rf":"dko_rf_7d", "dko_udder":"dko_udder_7d"})

ko_df3["dko_lr_nd_7d"] = (ko_df3["dko_lr_7d"]/7)*100
ko_df3["dko_lf_nd_7d"] = (ko_df3["dko_lf_7d"]/7)*100
ko_df3["dko_rf_nd_7d"] = (ko_df3["dko_rf_7d"]/7)*100
ko_df3["dko_rr_nd_7d"] = (ko_df3["dko_rr_7d"]/7)*100
ko_df3["dko_udder_nd_7d"] = (ko_df3["dko_udder_7d"]/7)*100

kicko2 = ko_df3

# kick off visit

## visits with kick offs current lactation 

In [6]:
ko_df = merged_df
ko_df.loc[:, "ko_udder"] = ((ko_df.ko_rr + ko_df.ko_rf +ko_df.ko_lr + ko_df.ko_lf) > 0 ).astype(int)
ko_df3 = ko_df.groupby(["animal_number", "lactation_number"]).agg({"ko_lr":"sum", "ko_lf":"sum", "ko_rr":"sum", "ko_rf":"sum", "ko_udder":"sum", "date":"count"}).reset_index()
ko_df3 = ko_df3.rename(columns ={"ko_lr": "vko_lr_cl", "ko_lf":"vko_lf_cl", "ko_rr":"vko_rr_cl", "ko_rf":"vko_rf_cl", "ko_udder":"vko_udder_cl", "date":"visits_cl"})

ko_df3["vko_lr_nv_cl"] = (ko_df3["vko_lr_cl"]/ko_df3['visits_cl'])*100
ko_df3["vko_lf_nv_cl"] = (ko_df3["vko_lf_cl"]/ko_df3['visits_cl'])*100
ko_df3["vko_rf_nv_cl"] = (ko_df3["vko_rf_cl"]/ko_df3['visits_cl'])*100
ko_df3["vko_rr_nv_cl"] = (ko_df3["vko_rr_cl"]/ko_df3['visits_cl'])*100
ko_df3["vko_udder_nv_cl"] = (ko_df3["vko_udder_cl"]/ko_df3['visits_cl'])*100

kicko3 = ko_df3

## visits with kick offs last 7d 

In [7]:
ko_df = merged_df[(merged_df.date >= '2023-11-10') & (merged_df.date < '2023-11-17')]
ko_df.loc[:, "ko_udder"] = ((ko_df.ko_rr + ko_df.ko_rf +ko_df.ko_lr + ko_df.ko_lf) > 0 ).astype(int)
ko_df3 = ko_df.groupby(["animal_number", "lactation_number"]).agg({"ko_lr":"sum", "ko_lf":"sum", "ko_rr":"sum", "ko_rf":"sum", "ko_udder":"sum", "date":"count"}).reset_index()
ko_df3 = ko_df3.rename(columns ={"ko_lr": "vko_lr_7d", "ko_lf":"vko_lf_7d", "ko_rr":"vko_rr_7d", "ko_rf":"vko_rf_7d", "ko_udder":"vko_udder_7d",  "date":"visits_7d"})

ko_df3["vko_lr_nv_7d"] = (ko_df3["vko_lr_7d"]/ko_df3['visits_7d'])*100
ko_df3["vko_lf_nv_7d"] = (ko_df3["vko_lf_7d"]/ko_df3['visits_7d'])*100
ko_df3["vko_rf_nv_7d"] = (ko_df3["vko_rf_7d"]/ko_df3['visits_7d'])*100
ko_df3["vko_rr_nv_7d"] = (ko_df3["vko_rr_7d"]/ko_df3['visits_7d'])*100
ko_df3["vko_udder_nv_7d"] = (ko_df3["vko_udder_7d"]/ko_df3['visits_7d'])*100

kicko4 = ko_df3

# Merge and save

In [8]:
integrated_df = kicko1.merge(kicko2, how='left', on = ["animal_number", "lactation_number"]).merge(kicko3, how='left', on = ["animal_number", "lactation_number"]).merge(kicko4, how='left', on = ["animal_number", "lactation_number"])
integrated_df.columns = [col.replace("rr", "rb").replace("lr", "lb").replace("animal_number", "cow") for col in integrated_df.columns]
integrated_df.to_csv(r"data_out\ko_integrated.csv", index = False)